In [131]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
import numpy as np
import pandas as pd

In [133]:
filename="../analysis_plateau0.95.xlsx"


In [134]:
current = pd.read_excel(filename)

In [135]:
epsilon = 0.05

In [136]:
def get_T_e_max(data, epsilon):
    return data.where(data['epsilon'] < epsilon).groupby(['d', 'M', 'noise']).agg(
        T_e_max = pd.NamedAgg(column="T", aggfunc="min"),
    ).reset_index()
def get_T_e_min(data, epsilon):
    return data.where(data['epsilon'] > epsilon).groupby(['d', 'M', 'noise']).agg(
        T_e_min = pd.NamedAgg(column="T", aggfunc="max"),
    ).reset_index()

In [137]:
maxLog2 = 6
candidates = pd.DataFrame(data={
    'd':[2 ** i for i in range(maxLog2 + 1)]
}).merge(
    pd.DataFrame(data={
        'M':[2 ** i for i in range(maxLog2 + 1)]
    }),
    how='cross',
).merge(
    pd.DataFrame(data={
        'noise':[0.1, 0.2]
    }),
    how='cross',
)

candidates

,d,M,noise
0,1,1,0.1
1,1,1,0.2
2,1,2,0.1
3,1,2,0.2
4,1,4,0.1
...,...,...,...
93,64,16,0.2
94,64,32,0.1
95,64,32,0.2
96,64,64,0.1


In [138]:
def get_next_tests(data, epsilon, candidates):
    data_min_max = get_T_e_max(data, epsilon).merge(
        get_T_e_min(data, epsilon),
        on=['d', 'M', 'noise'],
        how='outer',
    )

    temp1 = data_min_max[['d', 'M', 'noise', 'T_e_max']].dropna()
    temp1['T'] = (temp1['T_e_max'] / 2).astype('int')
    temp1 = temp1.drop(['T_e_max'], axis=1)

    temp2 = data_min_max[['d', 'M', 'noise', 'T_e_min']].dropna()
    temp2['T'] = (temp2['T_e_min'] * 2).astype('int')
    temp2 = temp2.drop(['T_e_min'], axis=1)

    temp3 = data_min_max.dropna()[['d', 'M', 'noise', 'T_e_max']]
    temp3['T'] = (temp3['T_e_max']).astype('int')
    temp3 = temp3.drop(['T_e_max'], axis=1)
    temp3['d'] *= 2

    temp4 = data_min_max.dropna()[['d', 'M', 'noise', 'T_e_max']]
    temp4['T'] = (temp4['T_e_max']).astype('int')
    temp4 = temp4.drop(['T_e_max'], axis=1)
    temp4['M'] *= 2

    explore = pd.concat([temp3, temp4]).groupby(['d', 'M', 'noise']).agg(
        T = pd.NamedAgg(column="T", aggfunc="max"),
    ).reset_index()
    candidates_with_data = data_min_max[ 
        (~data_min_max['T_e_max'].isna())
        | 
        (~data_min_max['T_e_min'].isna())
    ][['M', 'd', 'noise']]
    candidates_without_data = pd.concat([
        candidates_with_data, candidates_with_data, candidates
    ]).drop_duplicates(keep=False)
    explore = candidates_without_data.merge(
        explore, 
        left_on=['M', 'd', 'noise'],
        right_on=['M', 'd', 'noise'],
        how='inner',
    )
    
    result = pd.concat([temp1, temp2, explore]).drop_duplicates()

    temp = data[['d', "M", 'noise', 'T']].drop_duplicates()
    result = pd.concat([temp, temp, result]).drop_duplicates(
        keep=False
    ).sort_values([ 
        'noise', 'd', 'M', 'T'
    ])

    result = candidates.merge( 
        result, 
        left_on=['d', 'M', 'noise'],
        right_on=['d', 'M', 'noise'],
        how='left',
    ).dropna().drop_duplicates()
    return result

In [139]:
next_tests = get_next_tests( 
    current, 
    0.05, 
    candidates
)
next_tests['T'] = next_tests['T'].astype('int')
next_tests['patience'] = 24
next_tests['batch-size'] = 64
next_tests['epochs'] = 1500
next_tests['weight-decay'] = 0
next_tests['lr'] = 0.000
next_tests['dropout'] = 0
next_tests = next_tests.drop_duplicates()
next_tests['count'] = 1
next_tests['comment'] = 'reduce-lr'
next_tests['reduce-lr'] = 'T'

In [140]:
# next_tests

In [141]:
next_tests.sort_values('T').to_csv('config.tsv', sep='\t', index=False)